In [12]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn sqlite-utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.0 MB/s eta 0:00:00


In [9]:
!pip install gdown --upgrade


In [10]:
!gdown --folder "https://drive.google.com/drive/folders/1Aej6JmBaj_ojJXgnFwqScgHsKDdjrams" -O ./data


Retrieving folder contents
Processing file 1DjPrfiGjsYbIDRBsLWn7h9kmIJnz1Xkz olist_customers_dataset.csv
Processing file 1VXcrjSmmlW7WXhYfB1Ipb6CflPV9hYJL olist_geolocation_dataset.csv
Processing file 1qJxa_uVQfkFtn8Qps5Gqn1rOF5K8u-Pt olist_order_items_dataset.csv
Processing file 1wsARgl8H7FtaQDkNNCQMCyE98ZNUq4V3 olist_order_payments_dataset.csv
Processing file 1nZwWlyOHhTii-yKYOFmKbIOKPmySgOWN olist_order_reviews_dataset.csv
Processing file 1y5kxHgQ2jC4TY752wRJ4NYTbiIyTuoJT olist_orders_dataset.csv
Processing file 1UuNVgVJZ4oyd6NFHFWw5Yed715ODppxF olist_products_dataset.csv
Processing file 1DIACnVS6eCUGViWctnyEXDhPnd0XQ2qn olist_sellers_dataset.csv
Processing file 1JzzM6f3YvCWQJelk7g7MwRy1BCbr5y1s product_category_name_translation.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1DjPrfiGjsYbIDRBsLWn7h9kmIJnz1Xkz
To: /content/data/olist_customers_dataset.csv
100% 9.03M/9.03M 

In [18]:
import os
import pandas as pd

# Path where files are downloaded
data_path = "./data"

# Dictionary to store DataFrames
datasets = {}

# Loop through all files in the folder
for file in os.listdir(data_path):
    if file.endswith(".csv"):
        file_path = os.path.join(data_path, file)
        df_name = file.replace(".csv", "")  # name DataFrame by file name
        datasets[df_name] = pd.read_csv(file_path)
        print(f"Loaded {file} as dataframe '{df_name}' with shape {datasets[df_name].shape}")


Loaded olist_orders_dataset.csv as dataframe 'olist_orders_dataset' with shape (99441, 8)
Loaded olist_order_items_dataset.csv as dataframe 'olist_order_items_dataset' with shape (112650, 7)
Loaded olist_geolocation_dataset.csv as dataframe 'olist_geolocation_dataset' with shape (1000163, 5)
Loaded product_category_name_translation.csv as dataframe 'product_category_name_translation' with shape (71, 2)
Loaded olist_order_payments_dataset.csv as dataframe 'olist_order_payments_dataset' with shape (103886, 5)
Loaded olist_products_dataset.csv as dataframe 'olist_products_dataset' with shape (32951, 9)
Loaded olist_sellers_dataset.csv as dataframe 'olist_sellers_dataset' with shape (3095, 4)
Loaded olist_order_reviews_dataset.csv as dataframe 'olist_order_reviews_dataset' with shape (99224, 7)
Loaded olist_customers_dataset.csv as dataframe 'olist_customers_dataset' with shape (99441, 5)


In [22]:
import pandas as pd

# Load datasets from the ./data folder
orders = pd.read_csv("./data/olist_orders_dataset.csv")
customers = pd.read_csv("./data/olist_customers_dataset.csv")
items = pd.read_csv("./data/olist_order_items_dataset.csv")
payments = pd.read_csv("./data/olist_order_payments_dataset.csv")
products = pd.read_csv("./data/olist_products_dataset.csv")

# Merge core data
df = orders.merge(customers, on="customer_id") \
           .merge(items, on="order_id") \
           .merge(payments, on="order_id")

# Handle missing values
df = df.dropna()

# Feature engineering
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])
df["order_month"] = df["order_purchase_timestamp"].dt.to_period("M")

# Revenue per order
df["revenue"] = df["price"] + df["freight_value"]

# Customer lifetime value
customer_revenue = df.groupby("customer_unique_id")["revenue"].sum().reset_index()
customer_revenue.rename(columns={"revenue": "Customer_Lifetime_Value"}, inplace=True)

df = df.merge(customer_revenue, on="customer_unique_id")


In [23]:
import sqlite3
import pandas as pd

# Fix order_month column
df["order_month"] = df["order_purchase_timestamp"].dt.to_period("M").astype(str)

# Create SQLite database in memory
conn = sqlite3.connect(":memory:")

# Save dataframe to SQL table
df.to_sql("orders", conn, index=False, if_exists="replace")

# Top 10 cities by revenue
query1 = """
SELECT customer_city, SUM(revenue) AS total_revenue
FROM orders
GROUP BY customer_city
ORDER BY total_revenue DESC
LIMIT 10;
"""
result1 = pd.read_sql(query1, conn)
print("Top 10 Cities by Revenue:")
display(result1)

# Average order value per month
query2 = """
SELECT order_month, AVG(revenue) AS avg_order_value
FROM orders
GROUP BY order_month
ORDER BY order_month;
"""
result2 = pd.read_sql(query2, conn)
print("Average Order Value by Month:")
display(result2)

# Payment method popularity
query3 = """
SELECT payment_type, COUNT(*) AS num_payments, SUM(payment_value) AS total_payment
FROM orders
GROUP BY payment_type
ORDER BY total_payment DESC;
"""
result3 = pd.read_sql(query3, conn)
print("Payment Method Popularity:")
display(result3)


Top 10 Cities by Revenue:


,customer_city,total_revenue
0,sao paulo,2210046.65
1,rio de janeiro,1158079.58
2,belo horizonte,419524.64
3,brasilia,356981.07
4,curitiba,246379.39
5,salvador,226488.83
6,porto alegre,220524.47
7,campinas,218096.05
8,guarulhos,169758.59
9,niteroi,152101.87


Average Order Value by Month:


,order_month,avg_order_value
0,2016-10,145.350565
1,2016-12,19.620000
2,2017-01,142.179490
3,2017-02,145.418589
4,2017-03,142.970971
5,2017-04,152.883417
6,2017-05,143.276765
7,2017-06,139.295698
8,2017-07,128.417835
9,2017-08,133.636956


Payment Method Popularity:


...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(result3['payment_type'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(result3, x='total_payment', y='payment_type', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
result2['avg_order_value'].plot(kind='hist', bins=20, title='avg_order_value')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
result2['avg_order_value'].plot(kind='line', figsize=(8, 4), title='avg_order_value')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
result1['total_revenue'].plot(kind='hist', bins=20, title='total_revenue')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
result1['total_revenue'].plot(kind='line', figsize=(8, 4), title='total_revenue')
plt.gca().spines[['top', 'right']].set_visible(False)

In [24]:
df.to_csv("cleaned_orders.csv", index=False)
from google.colab import files
files.download("cleaned_orders.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>